# GEC Model Deployment

## Imports and Setup

In [21]:
import sys
import os
sys.path.append('../')

In [23]:
%%sh
# pip uninstall -y -q awscli
pip install awscli
pip install q -U sagemaker-ssh-helper
pip freeze | grep sagemaker-ssh-helper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 19.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 63.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.31.80
    Uninstalling botocore-1.31.80:
      Successfully uninstalled botocore-1.31.80


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.28.80 requires botocore<1.32.0,>=1.31.80, but you have botocore 1.32.6 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 91.3 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.32.6
    Uninstalling botocore-1.32.6:
      Successfully uninstalled botocore-1.32.6


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.30.6 requires botocore==1.32.6, but you have botocore 1.31.85 which is incompatible.


sagemaker-ssh-helper==2.1.0


In [24]:
%%writefile ssh_notebook.py

import time
import os

import sagemaker_ssh_helper
sagemaker_ssh_helper.setup_and_start_ssh()

while os.environ.get("START_SSH", "false") == "true":
    time.sleep(10)  # will sleep forever

Overwriting ssh_notebook.py


In [25]:
%%sh
LOCAL_USER_ID="AIDAYKGQHXPC6C3H7TY32"  # replace with your local UserId

if [ -f ~/.sm-ssh-owner ]; then
    echo "sm-ssh-notebook: SSH owner user ID is already set in ~/.sm-ssh-owner, skipping override"
else
    echo "sm-ssh-notebook: Saving SSH owner user ID into ~/.sm-ssh-owner"
    echo "$LOCAL_USER_ID" > ~/.sm-ssh-owner
fi


sm-ssh-notebook: SSH owner user ID is already set in ~/.sm-ssh-owner, skipping override


In [26]:
with open(os.path.expanduser('~/.sm-ssh-owner')) as f:
    local_user_id = f.readline().rstrip()

In [27]:
import sys
from utils import Config
import boto3
from sagemaker import get_execution_role
from model_utils import download_simple_model
import os
import boto3
import sagemaker
import pprint

from sagemaker_ssh_helper.wrapper import SSHEstimatorWrapper

pp = pprint.PrettyPrinter(indent=1)

role = sagemaker.get_execution_role()
print(role)
sess = sagemaker.Session()
sm_boto3 = boto3.client("sagemaker")
s3 = boto3.resource('s3')
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity()["Account"]

# The name of our algorithm -- i.e. the name of the inference container
INFERENCE_ALGORITHM_NAME = "sm-gec-aws"
ENDPOINT_NAME = "sm-gec-aws"
IMAGE_URI_INFERENCE = (
    f"{account_id}.dkr.ecr.{region}.amazonaws.com/{INFERENCE_ALGORITHM_NAME}:latest"
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
arn:aws:iam::571667364805:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


## Simple GEC - Inference Endpoint Configuration

### Imports And Setup

In [8]:
!pip install wandb tensorflow transformers

In [ ]:
!pip install --upgrade tensorflow sagemaker wandb

In [28]:
# Simple GEC specific constants
RUN_TO_DEPLOY = '3emhdbgu' # The wandb run from which to get model weights
S3_BUCKET = 'project-langbot-models'
S3_DATA_KEY = f'gec_simple_model_{RUN_TO_DEPLOY}_weights.gz'
PRETRAINED_MODEL_DATA = 's3://{}/{}'.format(S3_BUCKET, S3_DATA_KEY)

### Push Model Weights To S3 From WandB

In [21]:
# Upload the model weights from WandB to S3 bucket
def copy_weights_to_s3(runid, download=True):
    main_args = Config()
    run_ref = main_args.SIMPLE_MODEL_RUNS[runid]
    weights_filename = 'model_weights.gz'
    if download:
        download_simple_model(run_ref, weights_filename)
    # Push the downloaded weights to s3 bucket
    role = get_execution_role()
    s3.meta.client.upload_file(f'downloads/{weights_filename}', S3_BUCKET, S3_DATA_KEY)
    return

In [22]:
copy_weights_to_s3(RUN_TO_DEPLOY, download=True)

Found <Run langbot/langbot_gec_plain_top_performers/3emhdbgu (finished)> to load artifact from
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [46]:
# Confirm model weights are there on S3
print(PRETRAINED_MODEL_DATA)
s3.Bucket(S3_BUCKET).Object(S3_DATA_KEY)

s3://project-langbot-models/gec_simple_model_3emhdbgu_weights.gz


s3.Object(bucket_name='project-langbot-models', key='gec_simple_model_3emhdbgu_weights.gz')

### DockerFile

In [29]:
!cat model-simple/container/Dockerfile-inference

# Based on
# https://github.com/aws/amazon-sagemaker-examples/blob/main/advanced_functionality/tensorflow_iris_byom/tensorflow_BYOM_iris.ipynb

ARG REGION=us-west-2

# SageMaker TF image for INFERENCE
FROM 763104351884.dkr.ecr.$REGION.amazonaws.com/tensorflow-inference:2.12.1-gpu-py310-cu118-ubuntu20.04-sagemaker

ENV PATH="/opt/ml/code:${PATH}"

# /opt/ml and all subdirectories are utilized by SageMaker, we use the /code subdirectory to store our user code.
COPY /gec /opt/ml/code

# this environment variable is used by the SageMaker TensorFlow container to determine our user code directory.
ENV SAGEMAKER_SUBMIT_DIRECTORY /opt/ml/code

# this environment variable is used by the SageMaker Tensorflow container to determine our program entry point
# for training and serving.
# For more information: https://github.com/aws/sagemaker-tensorflow-training-toolkit
ENV SAGEMAKER_PROGRAM gec-simple-inference.py

RUN pip install --no-cache-dir --upgrade pip && \
    pip install --no-cache-dir prot

### Inference script - gec-simple-inference.py

In [30]:
!pygmentize model-simple/container/gec/gec-simple-inference.py

import logging
import os
import json
import pickle
import Config
import tensorflow as tf
from model_utils import create_simple_model, infer_simple


JSON_CONTENT_TYPE = 'application/json'

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

logger.info(f"====================== Running stuff ============================")

def _load_model_from_run_3emhdbgu(weights_loc):
    main_args = Config()
    train_config = main_args.bert_plain_models[0]
    optimizer = tf.keras.optimizers.AdamW(
        learning_rate=5e-5,
        weight_decay=0.005,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=False)

    train_config['optimizer'] = optimizer
    train_config['batch_size'] = 32
    train_config['train_layers'] = -1
    train_config['epochs'] = 10
    train_config['dropout_rate'] = 0.5
    train_config['load_from_pretrained'] = True
    train_config['export_weights'] = False
    train_config['weights_loc'] = weights_loc
    train_config['extra_

### Build and Push Container

In [31]:
!pwd

/home/ec2-user/SageMaker/model_gec_ram/deployment


In [57]:
! cd model-simple/container && sh build_and_push.sh {INFERENCE_ALGORITHM_NAME} Dockerfile-inference && cd ../..

ECR image fullname: 571667364805.dkr.ecr.us-west-2.amazonaws.com/sm-gec-aws:latest
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  19.97kB
Step 1/8 : ARG REGION=us-west-2
Step 2/8 : FROM 763104351884.dkr.ecr.$REGION.amazonaws.com/tensorflow-inference:2.12.1-gpu-py310-cu118-ubuntu20.04-sagemaker
 ---> fde65ca56ee6
Step 3/8 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Using cache
 ---> ca345d8b1c1c
Step 4/8 : COPY /gec /opt/ml/code
 ---> a45876d5d3de
Step 5/8 : ENV SAGEMAKER_SUBMIT_DIRECTORY /opt/ml/code
 ---> Running in 49c9

### Create Inference Endpoint

In [58]:
from sagemaker_ssh_helper.wrapper import SSHModelWrapper

In [59]:
from sagemaker import Model

import sagemaker as sage

sess = sage.Session()

# instance_type = "ml.m5.xlarge" # no GPU, will trigger an error
# instance_type = "ml.g4dn.xlarge"
instance_type = "ml.p2.xlarge"

# endpoint_deployment_name = "sm-gec-aws"

estimator = Model(
    image_uri=IMAGE_URI_INFERENCE,
    model_data=PRETRAINED_MODEL_DATA,
    role=role,
    source_dir="model-simple/container/gec",
    dependencies=[SSHModelWrapper.dependency_dir()],
    entry_point="gec-simple-inference.py",
    sagemaker_session=sess,  # not local session anymore
    #                   predictor_cls=None,
    #                   env=None,
    #                   name=None,
    #                   vpc_config=None,
    #                   enable_network_isolation=False,
    #                   model_kms_key=None,
    #                   image_config=None,
    #                   code_location=None,
    #                   container_log_level=20,
    #                   dependencies=None,
    #                   git_config=None
)

# ssh_wrapper = SSHEstimatorWrapper.create(estimator, connection_wait_time_seconds=0, local_user_id=local_user_id, log_to_stdout=True)

ssh_wrapper = SSHModelWrapper.create(estimator, connection_wait_time_seconds=0)

# deploy the model
predictor = estimator.deploy(1, instance_type, endpoint_name=ENDPOINT_NAME, wait=True)



sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
--------!

In [ ]:
instance_ids = ssh_wrapper.get_instance_ids(timeout_in_sec=30)

In [44]:
instance_ids

[]

## GEC With Explanation - Inference Endpoint Configuration

### Prerequisites

Pre trained NER model should be pickled and uploaded to the correct S3 bucket as a tarball. 

### Imports and Setup

In [1]:
BUCKET = "project-langbot-models"
KEY = "gec_cows_l2h_small.gz"
PRETRAINED_MODEL_DATA = "s3://{}/{}".format(BUCKET, KEY)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


### Check Saved Model

In [2]:
print(PRETRAINED_MODEL_DATA)
boto3.Session().resource("s3").Bucket(BUCKET).Object(KEY)

s3://project-langbot-models/gec_cows_l2h_small.gz


s3.Object(bucket_name='project-langbot-models', key='gec_cows_l2h_small.gz')

### Dockerfile

In [31]:
!cat model1/container/Dockerfile-inference

# Based on
# https://github.com/awslabs/amazon-sagemaker-examples/master/advanced_functionality/pytorch_extending_our_containers/pytorch_extending_our_containers.ipynb

ARG REGION=us-west-2

# SageMaker PyTorch image for INFERENCE
FROM 763104351884.dkr.ecr.$REGION.amazonaws.com/pytorch-inference:1.12.1-gpu-py38-cu113-ubuntu20.04-sagemaker

ENV PATH="/opt/ml/code:${PATH}"

# /opt/ml and all subdirectories are utilized by SageMaker, we use the /code subdirectory to store our user code.
COPY /gec /opt/ml/code

# this environment variable is used by the SageMaker PyTorch container to determine our user code directory.
ENV SAGEMAKER_SUBMIT_DIRECTORY /opt/ml/code

# this environment variable is used by the SageMaker PyTorch container to determine our program entry point
# for training and serving.
# For more information: https://github.com/aws/sagemaker-pytorch-container
ENV SAGEMAKER_PROGRAM gec-inference.py

RUN pip install --no-cache-dir --upgrade pip && \
    pip install --no-cache-dir n

### Inference script - gec-inference.py

In [4]:
!pygmentize container/gec/gec-inference.py

import logging
import os
import torch
import json
import pickle

JSON_CONTENT_TYPE = 'application/json'

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

logger.info(f"====================== Running stuff ============================")

def model_fn(model_dir):
    logger.info(f"inside model_fn, model_dir= {model_dir}")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    logger.info("Device Type: {}".format(device))

    model_loc = f'{model_dir}/gec_cows_l2h_small.pkl'
    if not os.path.exists(model_loc):
        logging.error(f'Missing model file {model_loc}')

    model = None
    with open(model_loc, 'rb') as f:  # open a text file
        model = pickle.load(f) # serialize the list
        # TODO: How to add GPU support for inferencing? Looks like NER model does not have .to() method.
        #       Also do we need GPU for inference?
        # model.to(device)

    logging.info(f'GEC model loaded into device {device}')
    return model

def predic

### Build and Push Container

In [5]:
! cd container && sh build_and_push.sh {INFERENCE_ALGORITHM_NAME} Dockerfile-inference

ECR image fullname: 571667364805.dkr.ecr.us-west-2.amazonaws.com/sm-gec-aws:latest
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  17.92kB
Step 1/8 : ARG REGION=us-west-2
Step 2/8 : FROM 763104351884.dkr.ecr.$REGION.amazonaws.com/pytorch-inference:1.12.1-gpu-py38-cu113-ubuntu20.04-sagemaker
 ---> cc486ae090f7
Step 3/8 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Using cache
 ---> ed6c9f68b5de
Step 4/8 : COPY /gec /opt/ml/code
 ---> Using cache
 ---> 923b5fcc86b5
Step 5/8 : ENV SAGEMAKER_SUBMIT_DIRECTORY /opt/ml/code
 ---> U

### Create Inference Endpoint

In [6]:
from sagemaker import Model

import sagemaker as sage

sess = sage.Session()

# instance_type = "ml.m5.xlarge" # no GPU, will trigger an error
# instance_type = "ml.g4dn.xlarge"
instance_type = "ml.p2.xlarge"

# endpoint_deployment_name = "sm-gec-aws"

estimator = Model(
    image_uri=IMAGE_URI_INFERENCE,
    model_data=PRETRAINED_MODEL_DATA,
    role=role,
    source_dir="container/gec",
    entry_point="gec-inference.py",
    sagemaker_session=sess,  # not local session anymore
    #                   predictor_cls=None,
    #                   env=None,
    #                   name=None,
    #                   vpc_config=None,
    #                   enable_network_isolation=False,
    #                   model_kms_key=None,
    #                   image_config=None,
    #                   code_location=None,
    #                   container_log_level=20,
    #                   dependencies=None,
    #                   git_config=None
)

# deploy the model
predictor = estimator.deploy(1, instance_type, endpoint_name=ENDPOINT_NAME)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
-------!

## Test Endpoint

In [56]:
import json
sm_client = sess.sagemaker_runtime_client
# endpoint_name = "sm-gec-aws"
response = sm_client.invoke_endpoint(
    EndpointName=ENDPOINT_NAME,
    ContentType="application/json",
    Body=json.dumps({"line":"Hola Como estas?"}),
)

r = response["Body"]
print("RESULT r.read().decode():", r.read().decode())

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2983/3261681033.py", line 4, in <module>
    response = sm_client.invoke_endpoint(
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/client.py", line 535, in _api_call
    return self._make_api_call(operation_name, kwargs)
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/client.py", line 963, in _make_api_call
    http, parsed_response = self._make_request(
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/client.py", line 986, in _make_request
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/endpoint.py", line 119, in make_request
    return self._send_request(request_dict, operation_

## Optional cleanup of the create endpoint
The created endpoint can be deleted with the code below.

This part represent the end of the notebook.

In [60]:
import boto3

client = boto3.client("sagemaker")
response = client.describe_endpoint_config(EndpointConfigName=ENDPOINT_NAME)
model_name = response["ProductionVariants"][0]["ModelName"]
client.delete_endpoint(EndpointName=ENDPOINT_NAME)
# client.delete_endpoint_config(EndpointConfigName=ENDPOINT_NAME)
# client.delete_model(ModelName=model_name)

{'ResponseMetadata': {'RequestId': '5c870347-dcb5-4c4f-9d5d-fa6fc62cb8f3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5c870347-dcb5-4c4f-9d5d-fa6fc62cb8f3',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Mon, 27 Nov 2023 00:03:29 GMT'},
  'RetryAttempts': 0}}